# 지금까지 배운거 복습

In [1]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch

# device setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Data download
mnist_train = dsets.MNIST(root = "MNIST_data/", train = True, transform = transforms.ToTensor(), download= True)
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False, transform = transforms.ToTensor(), download= True)

In [2]:
# hyper_parameter
batch_size = 100
learning_rate = 0.001
training_epochs = 10

In [3]:
# Dataloader
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset= mnist_train,
                          batch_size = batch_size,
                          shuffle= True,
                          drop_last= True)

test_loader = DataLoader(dataset= mnist_test,
                          batch_size = batch_size,
                          shuffle= True,
                          drop_last= False)


In [6]:
# model 만들기
import torch.nn as nn
class MnistClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            # layer1
            nn.Linear(28*28,256),
            nn.ReLU(),
            
            #layer2
            nn.Linear(256,256),
            nn.ReLU(),
            
            #layer3
            nn.Linear(256,256),
            nn.ReLU(),
            
            #layer4
            nn.Linear(256,10),
        )
        
    def forward(self,x):
        return self.layer(x)
model = MnistClassifier().to(device)

In [7]:
# define cost/loss & optim
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer= torch.optim.Adam(model.parameters(), lr = learning_rate)


In [13]:
# train & test
total_batch = len(train_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    correct = 0
    
    for X,Y in train_loader:
        X = X.view(-1,28*28)
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        # backward
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        #calculate
        avg_cost += cost.item()
        correct += (torch.argmax(hypothesis, 1).float() == Y).sum().item()
        
    acc = correct/(total_batch*batch_size)*100
    avg_cost /= total_batch
    print("Epoch: {:04d} cost = {:.4f} Acc = {:.4f} %".format(epoch+1, avg_cost, acc))
    
    # test
    with torch.no_grad():
        correct_prediction = 0
        for X_test,Y_test in test_loader:
            X_test = X_test.view(-1,28*28)
            prediction = model(X_test)
            correct_prediction += torch.argmax(prediction,1) == Y_test
        accuarcy = correct_prediction.float().mean()
        print('Test Accuracy: {:.4f}'.format(accuarcy.item()))
            
        

Epoch: 0001 cost = 0.0125 Acc = 99.6233 %
Test Accuracy: 98.0900
Epoch: 0002 cost = 0.0135 Acc = 99.6000 %
Test Accuracy: 98.2100
Epoch: 0003 cost = 0.0134 Acc = 99.5983 %
Test Accuracy: 98.0000
Epoch: 0004 cost = 0.0120 Acc = 99.6067 %
Test Accuracy: 97.5800
Epoch: 0005 cost = 0.0121 Acc = 99.6167 %
Test Accuracy: 97.7700
Epoch: 0006 cost = 0.0074 Acc = 99.7483 %
Test Accuracy: 98.2300
Epoch: 0007 cost = 0.0138 Acc = 99.5583 %
Test Accuracy: 97.9200
Epoch: 0008 cost = 0.0107 Acc = 99.6567 %
Test Accuracy: 98.0600
Epoch: 0009 cost = 0.0060 Acc = 99.7983 %
Test Accuracy: 98.1200
Epoch: 0010 cost = 0.0069 Acc = 99.7817 %
Test Accuracy: 97.8700


In [25]:
t = torch.FloatTensor([[1,2],
                       [3,4],
                       [5,6]])

t.max(dim = 0)


torch.return_types.max(
values=tensor([5., 6.]),
indices=tensor([2, 2]))